# Options Pricing
This Jupyter notebook contains the enitre implementation of my options pricing model.

In [3]:
# import the necessary libaries
import math
import numpy as np
from scipy.stats import norm, mvn

## Overview
This project is intended to price a type of financial product called [options](https://www.investopedia.com/terms/o/option.asp). These are a type of [financial derivative](https://www.investopedia.com/terms/d/derivative.asp). The models are mostly different variations of the [Black-Scholes-Merton (BSM) model](https://www.investopedia.com/terms/b/blackscholes.asp) that is used to price [European options](https://www.investopedia.com/terms/e/europeanoption.asp). This notebook also contains approximations to value [American options](https://www.investopedia.com/terms/a/americanoption.asp#:~:text=An%20American%20option%2C%20aka%20an,including%20the%20day%20of%20expiration.).

### Option Pricing Models
| Pricing Models    | Description                               |
| ----------------- | ----------------------------------------- |
| BlackScholes()    | Stock options (no dividend yield)         |
| Merton()          | Index options (continuous dividend yield) |
| Black76()         | Commodity options                         |
| GarmanKohlhagen() | FX options                                |
| Asian76()         | Asian options (on commodities)            |
| Kirks76()         | Spread options (Kirk's approximation)     |
| American()        | American options                          |
| American76()      | American options (on commodities)         |

### Implied Volatility Calculations
| Implied Volatility Formula | Description                       |
| -------------------------- | --------------------------------- |
| EuroIV()                   | European options                  |
| EuroIV76()                 | European options (on commodities) |
| AmerIV()                   | American options                  |
| AmerIV76()                 | American options (on commodities) |